In [1]:
!pip install huggingface_hub mamba_ssm torch datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_c

In [2]:
!wget "https://huggingface.co/datasets/alberto98fx/Letteratura-Italiana/resolve/main/full-texts.txt"

--2024-08-09 18:00:08--  https://huggingface.co/datasets/alberto98fx/Letteratura-Italiana/resolve/main/full-texts.txt
Resolving huggingface.co (huggingface.co)... 13.33.30.49, 13.33.30.76, 13.33.30.23, ...
Connecting to huggingface.co (huggingface.co)|13.33.30.49|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/8d/33/8d338a1174ed56cdf26d6f35dde3975d404135392aa367e7c496d94ba3dac472/6c99c9d7a5f8407955ff1cc474f78cfb0cc7fab9129776ef8cc60e0e6428645b?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27full-texts.txt%3B+filename%3D%22full-texts.txt%22%3B&response-content-type=text%2Fplain&Expires=1723485608&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyMzQ4NTYwOH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzhkLzMzLzhkMzM4YTExNzRlZDU2Y2RmMjZkNmYzNWRkZTM5NzVkNDA0MTM1MzkyYWEzNjdlN2M0OTZkOTRiYTNkYWM0NzIvNmM5OWM5ZDdhNWY4NDA3OTU1ZmYxY2M0NzRmNzh

In [3]:
!wget "https://huggingface.co/alberto98fx/Italian-Poet-Mamba/resolve/main/model.pt"

--2024-08-09 18:00:09--  https://huggingface.co/alberto98fx/Italian-Poet-Mamba/resolve/main/model.pt
Resolving huggingface.co (huggingface.co)... 13.33.30.49, 13.33.30.76, 13.33.30.23, ...
Connecting to huggingface.co (huggingface.co)|13.33.30.49|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/7b/eb/7beba195ca3f73a1354c8053ff59a6ce11c8170c300a30ff7aad91c0bf54c4bd/a81295ac8523da7391567971ac45ceaba2aa9299a258b2d97c96dff8d5d65f36?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.pt%3B+filename%3D%22model.pt%22%3B&Expires=1723484418&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyMzQ4NDQxOH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzdiL2ViLzdiZWJhMTk1Y2EzZjczYTEzNTRjODA1M2ZmNTlhNmNlMTFjODE3MGMzMDBhMzBmZjdhYWQ5MWMwYmY1NGM0YmQvYTgxMjk1YWM4NTIzZGE3MzkxNTY3OTcxYWM0NWNlYWJhMmFhOTI5OWEyNThiMmQ5N2M5NmRmZjhkNWQ2NWYzNj9yZXNwb25zZS1jb25

In [4]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from mamba_ssm import Mamba

# Function to determine vocab size from the model file
def get_vocab_size(model_path):
    model = torch.load(model_path, map_location=torch.device('cpu'))

    def find_embedding_layer(model_dict):
        for key, value in model_dict.items():
            if isinstance(value, torch.Tensor) and len(value.shape) == 2:
                if key.endswith('weight') and 'embedding' in key.lower():
                    return value
        return None

    embedding_layer = find_embedding_layer(model)

    if embedding_layer is not None:
        return embedding_layer.shape[0]
    else:
        raise ValueError("Could not determine vocabulary size. The model structure might be different.")

# Hyperparameters (these should match the ones used during training)
block_size = 256
n_embed = 384
n_heads = 6
n_layers = 6
dropout = 0.2

# Model architecture
class SelfAttentionHead(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.keys = nn.Linear(n_embed, head_size)
        self.queries = nn.Linear(n_embed, head_size)
        self.values = nn.Linear(n_embed, head_size)
        self.head_size = head_size
        self.n_embed = n_embed
        self.register_buffer('tril', torch.tril(torch.ones((block_size, block_size))))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.keys(x)
        q = self.queries(x)
        v = self.values(x)
        wei = k @ q.transpose(-1, -2) * C ** (-0.5)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = torch.log(torch.exp(wei) + 1)
        wei = self.dropout(wei)
        out = wei @ v
        return out

class LayerNorm(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.eps = 1e-5
        self.gamma = nn.Parameter(torch.ones(dim))
        self.beta = nn.Parameter(torch.zeros(dim))

    def forward(self, x):
        xmean = x.mean(dim=1, keepdim=True)
        xvar = ((x - xmean) ** 2).mean(dim=1, keepdim=True)
        xhat = (x - xmean) / torch.sqrt(xvar + self.eps)
        self.out = self.gamma * xhat + self.beta
        return self.out

class MultiHeadAttention(nn.Module):
    def __init__(self, n_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([SelfAttentionHead(head_size) for _ in range(n_heads)])
        self.proj = nn.Linear(n_embed, n_embed)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([head(x) for head in self.heads], dim=-1)
        out = self.proj(out)
        out = self.dropout(out)
        return out

class FeedForward(nn.Module):
    def __init__(self, n_embed):
        super().__init__()
        self.ffn = nn.Sequential(
            nn.Linear(n_embed, 4 * n_embed),
            nn.ReLU(),
            nn.Linear(4 * n_embed, n_embed),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.ffn(x)

class Block(nn.Module):
    def __init__(self, n_embed, n_heads):
        super().__init__()
        self.head_size = n_embed // n_heads
        self.sa_head = Mamba(
            d_model=n_embed,
            d_state=16,
            d_conv=4,
            expand=2,
        )
        self.ffn = FeedForward(n_embed)
        self.ln1 = nn.LayerNorm(n_embed)
        self.ln2 = nn.LayerNorm(n_embed)

    def forward(self, x):
        x = x + self.sa_head(self.ln1(x))
        x = x + self.ffn(self.ln2(x))
        return x

class BigramNeuralNetwork(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
        self.position_embedding_table = nn.Embedding(block_size, n_embed)
        self.sa_head = MultiHeadAttention(4, int(n_embed / 4))
        self.lm_head = nn.Linear(n_embed, vocab_size)
        self.ffn = FeedForward(n_embed)
        self.blocks = nn.Sequential(*[Block(n_embed, n_heads=n_heads) for _ in range(n_layers)])

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=idx.device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        logits = self.lm_head(x)
        return logits, None

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, _ = self(idx_cond)
            last_timestep = logits[:, -1, :]
            probs = F.softmax(last_timestep, dim=1)
            next_index = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, next_index), dim=1)
        return idx

# Determine vocab size and load the model
model_path = 'model.pt'
vocab_size = get_vocab_size(model_path)
print(f"Determined vocabulary size: {vocab_size}")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BigramNeuralNetwork(vocab_size).to(device)
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()

# Encoding and decoding functions
chars = sorted(list(set(open('full-texts.txt', 'r').read())))
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}

def encode(s):
    return [stoi[c] for c in s]

def decode(l):
    return ''.join([itos[i] for i in l])

# Generate text
seed_text = "Ricordo la farfalla ch’era entrata dai vetri schiusi nella sera fumida su la costa raccolta, dilavata dal trascorrere iroso delle spume."
encoded_seed = encode(seed_text)
seed_tensor = torch.tensor(encoded_seed, dtype=torch.long, device=device).unsqueeze(0)

generated_indices = model.generate(seed_tensor, max_new_tokens=500)[0].tolist()
generated_text = decode(generated_indices)

print("\nGenerated text:")
print(generated_text)


Determined vocabulary size: 151

Generated text:
Ricordo la farfalla ch’era entrata dai vetri schiusi nella sera fumida su la costa raccolta, dilavata dal trascorrere iroso delle spume.

I soliti corsari erano, giunti insieme al notaio, quando videro i mozzi, si videro correre precipitosamente contro le vicinanze dell’avvocato Arzellini, il quale nessuna poteva fare prima del rimorso a Firenze, dirigendosi verso il suo marito.

Poi, finita quella prima volta a sua volta, pur sicuro di averlo.





I.





Elegante ordine sapeva a mia moglie un grasso sudicenziario per azione ch’ella aveva consentito al mio rispetto per un maschio e tante parole gli avevano ingiusta veramente. 
